In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import math
import numpy
from datetime import datetime
from shapely import wkt
from shapely.errors import WKTReadingError
from shapely.geometry import mapping, shape
from shapely import Polygon
import warnings
warnings.filterwarnings("ignore")
import re
import numpy as np

In [ ]:
login = {
    "username":"Dorito",
    "password":"Portador123"
}
# Crear una sesión para mantener la cookie
session = requests.Session()
response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)

In [ ]:
def get_data_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)
    datos = []
    while url:
        response = session.get(url, verify=False)
        info = response.json()
        datos.extend(info["results"])
        url = info["next"]
    return datos

def get_data_not_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)
    response = session.get(url, verify=False)
    info = response.json()
    return info


def parse_geom_safe(wkt_string):
    try:
        if isinstance(wkt_string, str) and "SRID=" in wkt_string:
            return wkt.loads(wkt_string.split(";", 1)[1])
    except:
        pass
    return None  # simplemente ignora lo inválido

In [ ]:
# Obtener y normalizar datos
url = "https://192.168.179.3/api/v1/poligonos_lotes/"
url_productor = "https://192.168.179.3/api/v1/productor/"
url_productor_2 = "https://192.168.179.3/api/v1/productor/mongo_get/productor_data/"
url_fincas = "https://192.168.179.3/api/v1/poligonos_fincas/"
url_lotes = "https://192.168.179.3/api/v1/poligonos_lotes/mongo_get/mongo_atribute/"
print("cargando datos lotes")
datos = get_data_paginated(url)
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos productor")
datos_productor_2 = get_data_not_paginated(url_productor_2)
print("cargando datos fincas")
datos_fincas = get_data_paginated(url_fincas)
print("cargando datos lotes")
datos_lotes = get_data_not_paginated(url_lotes)

df = pd.json_normalize(datos)
df_productor = pd.json_normalize(datos_productor)
df_productor_2 = pd.json_normalize(datos_productor_2)
df_fincas = pd.json_normalize(datos_fincas)
df_lotes = pd.json_normalize(datos_lotes)


In [ ]:
# Filtrar productores que tienen postgres_data.id
df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df.merge(
    df_productor,
    left_on="productor",
    right_on="id",
    how="left"
)
df_completo = df_completo.merge(
    df_productor_2,
    left_on="productor",
    right_on="postgres_data.id",
    how="left"
)

df_completo = df_completo.merge(
    df_fincas,
    left_on="finca",
    right_on="id",
    how="left"
)

df_completo = df_completo.merge(
    df_lotes,
    left_on="id_x",
    right_on="poligono_id",
    how="left"
)

# Aplicar solo a las válidas
df_completo["geometry"] = df_completo["poligono_x"].apply(parse_geom_safe)

# Crear el GeoDataFrame sin que explote
gdf_lotes = gpd.GeoDataFrame(df_completo, geometry="geometry", crs="EPSG:4326")


In [ ]:
# Obtener y normalizar datos
url = "https://192.168.179.3/api/v1/poligonos_fincas/"
url_productor = "https://192.168.179.3/api/v1/productor/"
url_productor_2 = "https://192.168.179.3/api/v1/productor/mongo_get/productor_data/"
url_lotes = "https://192.168.179.3/api/v1/poligonos_fincas/mongo_get/mongo_atribute/"
print("cargando datos fincas")
datos = get_data_paginated(url)
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos productor_2")
datos_productor_2 = get_data_not_paginated(url_productor_2)
print("cargando datos fincas atributos")
datos_lotes = get_data_not_paginated(url_lotes)

df = pd.json_normalize(datos)
df_productor = pd.json_normalize(datos_productor)
df_productor_2 = pd.json_normalize(datos_productor_2)
df_lotes = pd.json_normalize(datos_lotes)

In [ ]:
# Filtrar productores que tienen postgres_data.id
df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df.merge(
    df_productor,
    left_on="productor",
    right_on="id",
    how="left"
)

df_completo = df_completo.merge(
    df_productor_2,
    left_on="productor",
    right_on="postgres_data.id",
    how="left"
)

df_completo = df_completo.merge(
    df_lotes,
    left_on="id_x",
    right_on="poligono_id",
    how="left"
)

# Aplicar solo a las válidas
df_completo["geometry"] = df_completo["poligono"].apply(parse_geom_safe)

# Crear el GeoDataFrame sin que explote
gdf = gpd.GeoDataFrame(df_completo, geometry="geometry", crs="EPSG:4326")

In [ ]:
df = pd.read_excel(r"C:\Users\dorito\Downloads\HABILIDAD OCTUBRE 2025 COOCENTRAL(3.626) - UNIDAD TECNICA.xlsx", sheet_name="OCTUBRE")

In [ ]:
LISTA_CEDULAS = df["CEDULA"].to_list()

In [ ]:
gdf = gdf.loc[gdf["documento"].isin(LISTA_CEDULAS)]
gdf = gdf.sort_values("area", ascending=False)
gdf["geometry"] = gdf["geometry"].buffer(0)


In [ ]:
gdf_lotes.columns

In [ ]:
gdf.columns

In [ ]:
from shapely.ops import unary_union
import geopandas as gpd
import pandas as pd

# Parámetros que puedes ajustar
PROJ_CRS = 3116            # CRS métrico para área (usa el que sea correcto para tu región)
MIN_INTERSECTION_RATIO = 0.70   # proporción mínima para considerar pertenencia
MIN_ABS_INTERSECTION_AREA_M2 = 10  # área mínima absoluta en m² para descartar intersecciones puntuales/liniales
SHRINK_EPS = 0.5           # metros para "encoger" polígonos y evitar contactos por vértice (ajustable)

# --- Preprocesamiento: trabajar en proyección métrica para todas las operaciones de área/overlay
gdf_proj = gdf.to_crs(PROJ_CRS).copy()
gdf_lotes_proj = gdf_lotes.to_crs(PROJ_CRS).copy()

# Para poder referir a atributos originales (si necesitas) mantén el gdf original también
gdf_orig = gdf.copy()

ids_evaluados = set()
respuesta = []
poligonos_fincas_duplicados = []
counter = 1
n = len(gdf_proj)

for idx, row in gdf_proj.iterrows():
    print(f"{counter}/{n}")
    counter += 1

    id_actual = row["id_x"]
    cedula_actual = row["documento"]

    if id_actual in ids_evaluados:
        continue

    poly_head = row["geometry"]
    if poly_head is None or poly_head.is_empty:
        continue

    # opcional: encoger un poco para evitar que contactos por vértice se consideren intersección real
    try:
        poly_head_shrunk = poly_head.buffer(-SHRINK_EPS)
        if poly_head_shrunk.is_empty:
            poly_head_shrunk = poly_head  # fallback si el shrink deja vacío
    except Exception:
        poly_head_shrunk = poly_head

    area_head = poly_head.area  # en m²
    if area_head <= 0:
        continue

    ids_familiares = []
    ids_evaluados.add(id_actual)

    # Recorremos otros polígonos (en la versión proyectada)
    for idx2, row2 in gdf_proj.iterrows():
        id2 = row2["id_x"]
        if id2 == id_actual or id2 in ids_evaluados:
            continue

        poly2 = row2["geometry"]
        if poly2 is None or poly2.is_empty:
            continue

        # también intentar shrink en el otro para evitar contactos puntuales
        try:
            poly2_shrunk = poly2.buffer(-SHRINK_EPS)
            if poly2_shrunk.is_empty:
                poly2_shrunk = poly2
        except Exception:
            poly2_shrunk = poly2

        # Si no se intersectan geométricamente, skip rápido
        if not poly_head_shrunk.intersects(poly2_shrunk):
            continue

        inter = poly_head_shrunk.intersection(poly2_shrunk)
        if inter.is_empty:
            continue

        # Ignorar intersecciones que son solo punto o línea (contactos)
        if inter.geom_type in ("Point", "MultiPoint", "LineString", "MultiLineString"):
            continue

        inter_area = inter.area  # en m²
        # Descartar intersecciones demasiado pequeñas (tolerancia absoluta)
        if inter_area < MIN_ABS_INTERSECTION_AREA_M2:
            continue

        # Proporción: comparo con el área del polígono *más pequeño* para evitar falsos si uno es mucho más grande
        area_poly2 = poly2.area
        denom = min(area_head, area_poly2) if min(area_head, area_poly2) > 0 else max(area_head, area_poly2)
        pct = inter_area / denom

        # Considerar también contains (pero usando geometrías no-shrunk para mayor robustez)
        contains_check = poly_head.contains(poly2)

        if pct >= MIN_INTERSECTION_RATIO or contains_check:
            ids_evaluados.add(id2)
            ids_familiares.append(id2)

    # Procesar familia si se encontró al menos 1 miembro
    if len(ids_familiares) > 0:
        # Tomamos familiares que no sean la misma cédula (igual que antes)
        gdf_familiares = gdf_orig.loc[(gdf_orig["id_x"].isin(ids_familiares)) & (gdf_orig["documento"] != cedula_actual)]

        if len(gdf_familiares) == 0:
            gdf_familiares = gdf_orig.loc[gdf_orig["id_x"].isin(ids_familiares)]
            gdf_familia_total = gpd.GeoDataFrame(pd.concat([gdf_orig.loc[gdf_orig["id_x"]==id_actual], gdf_familiares]), crs=gdf_orig.crs)
            poligonos_fincas_duplicados.append(gdf_familia_total)
            continue

        # Unificar cabeza + familiares (original CRS para almacenamiento)
        gdf_cabeza_familiar = gdf_orig.loc[gdf_orig["id_x"] == id_actual]
        gdf_familia_total = gpd.GeoDataFrame(pd.concat([gdf_cabeza_familiar, gdf_familiares]), crs=gdf_orig.crs)

        # Calcular área total de la familia en CRS métrico
        gdf_familia_total_proj = gdf_familia_total.to_crs(PROJ_CRS)
        poligono_total = unary_union(gdf_familia_total_proj["geometry"])
        area_total_ha = poligono_total.area / 10000  # hectáreas

        # Deduplicar por documento y crear lista de miembros con índice limpio
        gdf_familia_total_unique = gdf_familia_total.drop_duplicates(subset="documento").reset_index(drop=True)

        miembros_texto = "\n".join(
            f"{i}. {doc} - {nom}"
            for i, (doc, nom) in enumerate(zip(gdf_familia_total_unique["documento"], gdf_familia_total_unique["datos.nombre_completo"]), start=1)
        )

        miembros = []
        for idx_m, row_m in gdf_familia_total_unique.reset_index(drop=True).iterrows():
            miembros.append({
                "indice": idx_m + 1,
                "documento": row_m["documento"],
                "codigo_finca": row_m.get("codigo_finca", None)
            })

        # Construir estructura base
        data_familia = {
            "cabeza_familia": f"{gdf_cabeza_familiar['documento'].iloc[0]} - {gdf_cabeza_familiar['datos.nombre_completo'].iloc[0]}",
            "numero_productores_en_familia": len(gdf_familia_total),
            "area_finca_familiar_ha": area_total_ha,
            "miembros_familiares": miembros_texto
        }

        # Para calcular el área por miembro usamos gdf_lotes_proj (CRS métrico)
        df_miembros = pd.DataFrame(miembros)
        for _, mrow in df_miembros.iterrows():
            documento_familiar = mrow["documento"]
            codigo_finca = mrow["codigo_finca"]

            # Filtrado en proyectado
            gdf_lotes_filtro = gdf_lotes_proj.loc[
                (gdf_lotes_proj["documento"] == documento_familiar) &
                (gdf_lotes_proj["codigo_finca"] == codigo_finca)
            ]

            if gdf_lotes_filtro.empty:
                area_finca_ha = 0.0
            else:
                pol_miembro = unary_union(gdf_lotes_filtro.geometry)
                if pol_miembro.is_empty:
                    area_finca_ha = 0.0
                else:
                    area_finca_ha = pol_miembro.area / 10000  # hectáreas

            # Nombre de columna claro y seguro
            col_name = f"miembro_{int(mrow['indice'])}_area_ha"
            data_familia[col_name] = area_finca_ha

        respuesta.append(data_familia)

print("Procesamiento finalizado.")


In [ ]:
df = pd.DataFrame(respuesta)

In [ ]:
df.to_excel("./Familias Coocentral.xlsx")